In [1]:
import os
import pandas as pd
import arcpy
import arcpy.mp as mp
import re
import requests
import numpy as np
import geopandas
from arcpy.sa import *
arcpy.CheckOutExtension("Spatial")
import random
import string

#### Get the path of this notebook and use it to generate path for input shapefile:

In [2]:
current_dir = os.getcwd()
shapefile_cbg = r"input\cbg_kontur.shp"
shapefile_cbg_path = os.path.join(current_dir, shapefile_cbg)
output_folder_path = os.path.join(current_dir, "output")
print(f"Location of this notebook: {current_dir}")
print(f"Location of CBG SHP: {shapefile_cbg_path}")
print(f"Location for output: {output_folder_path}")

Location of this notebook: C:\GITHUB\CCSVI\Scripts\Spatial_Analysis
Location of CBG SHP: C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\input\cbg_kontur.shp
Location for output: C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\output


#### Set up ArcPy Workspace:

In [3]:
arcpy.env.overwriteOutput = True
workspace = os.path.dirname(shapefile_cbg_path)
arcpy.env.workspace = workspace

#### Load shapefile into a layer and check spatial reference etc:

In [4]:
cbg_layer = shapefile_cbg_path
desc = arcpy.Describe(cbg_layer)
fields = [f.name for f in arcpy.ListFields(cbg_layer)]
count = arcpy.management.GetCount(cbg_layer)
print("Layer cbg_layer info:\n")
print("Shape type:", desc.shapeType)
print("Spatial reference:", desc.spatialReference.name)
print(f"Number of features: {count[0]}\n")
print(f"Field List: {fields}")


Layer cbg_layer info:

Shape type: Polygon
Spatial reference: WGS_1984_Web_Mercator_Auxiliary_Sphere
Number of features: 1057

Field List: ['FID', 'Shape', 'GEOID', 'GEO_ID', 'GEOIDFQ', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'NAMELSAD', 'P1_001N', 'trim_acres', 'pop_per_ac', 'pop_sqkm', 'Shape_Leng', 'Shape_Area', 'whole_bg_a', 'whole_bg_s', 'trim_bg_ac', 'trim_bg_sq', 'rainavgin']


#### Define location/name of raster for analysis and fieldname to put resulting value into:

In [5]:
#raster_path = os.path.join(workspace, r"environmental\staterf_inann.tif")
raster_path = os.path.join(workspace, r"environmental\igtn_prob_test.tif")
stat_type = "MAXIMUM"  # other options: "MEAN", "MAX", "MIN", "SUM", etc.

# Create field name using raster base name and stat type
raster_name = os.path.splitext(os.path.basename(raster_path))[0]
output_field = f"{raster_name}_{stat_type.lower()}"


#### Do Zonal Statistics and put output into DBF named similarly to the raster in an OUTPUT folder:

In [6]:
output_folder = os.path.join(current_dir, "output")
os.makedirs(output_folder, exist_ok=True)

In [7]:
# Zonal stats setup
zone_field_ident = "GEOIDFQ"
out_table_path = os.path.join(output_folder, f"{raster_name}_{stat_type.lower()}.dbf")

arcpy.sa.ZonalStatisticsAsTable(
    in_zone_data=cbg_layer,
    zone_field=zone_field_ident,
    in_value_raster=raster_path,
    out_table=out_table_path,
    statistics_type=stat_type,
    ignore_nodata="DATA"
)


<Result 'C:\\GITHUB\\CCSVI\\Scripts\\Spatial_Analysis\\output\\igtn_prob_test_maximum.dbf'>

#### Convert DBF to CSV: